## Part 1. Get metadata

In [1]:
import re
import sys
import pandas as pd
import urllib2 
from bs4 import BeautifulSoup 
import json

In [2]:
import csv
import requests 

In [3]:
from fake_useragent import UserAgent
import requests
from time import sleep

In [4]:
ua = UserAgent()
# print(ua.chrome)
header = {'User-Agent':str(ua.chrome)}
print(header)

{'User-Agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1866.237 Safari/537.36'}


In [116]:
f = open("C:\Users\mariia.petryk\Downloads\sage_counterfeit.ris","r")
data = f.readlines()

In [117]:
j=0
k=0
batch_list=list()
auth=str("")
df=pd.DataFrame(columns=['Author', 'Title','Pub_type', 'Journal', 'Year_publish', 'Doi_link','Abstract','Keywords'])
for i in range(0,len(data)):
# for i in range(0,20):
#     print i,j
    if not data[i].startswith('UR'):
#         print data[i].startswith('UR')
        if data[i].startswith('T1'):
            title=data[i][data[i].find("-")+2:].strip('\n')
#             print title
            df['Title'].loc[j] =title
        if data[i].startswith('T2'):
            jour=data[i][data[i].find("-")+2:].strip('\n')
#             print jour
            df['Journal'].loc[j] =jour
        if data[i].startswith('PY'):
            year=data[i][data[i].find("-")+2:].strip('\n')
#             print year
            df['Year_publish'].loc[j] =year
        if data[i].startswith('N2'):
            abstr=data[i][data[i].find("-")+2:].strip('\n')
            df['Abstract'].loc[j] =abstr
        j=j
    else:
        doi_link=data[i][data[i].find("-")+2:].strip('\n')
        df['Doi_link'].loc[j] =doi_link
        n=data.index(data[i])
#         print n
        batch=data[0+k+1:n+1]
#         print batch
        for l in range(0,len(batch)):
            if batch[l].startswith('AU'):
                a=batch[l][batch[l].find("-")+2:].strip('\n')
                if len(auth)==0:
                    auth=a
                else:
                    auth=auth+","+a
#         print auth
        df['Author'].loc[j] =auth
        auth=str("")
        k=n
        j=j+1    

In [118]:
# Append columns into one dataframe
df = pd.concat([df['Author'],  df['Title'],df['Pub_type'],df['Journal'],df['Year_publish'],df['Doi_link'],df['Abstract']], axis=1)

In [119]:
df.head()

,Author,Title,Pub_type,Journal,Year_publish,Doi_link,Abstract
0,"Greenberg, Mary Anne",Serialization: Benefits Beyond Regulatory Comp...,NaN,Therapeutic Innovation & Regulatory Science,2014,https://doi.org/10.1177/2168479014553034,As pharmaceutical and biotech markets continue...
1,"Isles, Mike",What's in a Word? Falsified/Counterfeit/Fake M...,NaN,Medicine Access @ Point of Care,2017,https://doi.org/10.5301/maapoc.0000008,There is a rising tide of criminal activity to...
2,"Carter, Alan",Improving the Drug Quality and Safety Net,NaN,Journal of Diabetes Science and Technology,2014,https://doi.org/10.1177/1932296814531100,"When large quantities of contaminated, subpote..."
3,"Lavorgna, Anita",The online trade in counterfeit pharmaceutical...,NaN,European Journal of Criminology,2014,https://doi.org/10.1177/1477370814554722,There is widespread agreement that the Interne...
4,"Rojek, Chris","Counterfeit Commerce: Relations of Production,...",NaN,Cultural Sociology,2016,https://doi.org/10.1177/1749975516650233,Study of the consumption of counterfeit produc...


In [120]:
# Get keywords for each paper
import requests
url22=df['Doi_link']
kw=pd.DataFrame(columns=['Keywords'])

In [121]:
len(df)

34

In [122]:
i=0
for i,link in url22.iteritems():  
    url = url22[i]
    print i,url
    try:
        f=requests.get(url, stream = True, headers=header)
        text=f.text
        soup = BeautifulSoup(text, "lxml")
        gg=soup.findAll("div", {"class": "abstractKeywords"})
        text=gg[0].get_text()
        t1=text.encode('ascii','ignore')
        tf=t1.replace('\n','')
        kw['Keywords'].loc[i]=tf
    except:
        kw['Keywords'].loc[i]=''
    sleep(5)

0 https://doi.org/10.1177/2168479014553034
1 https://doi.org/10.5301/maapoc.0000008
2 https://doi.org/10.1177/1932296814531100
3 https://doi.org/10.1177/1477370814554722
4 https://doi.org/10.1177/1749975516650233
5 https://doi.org/10.1177/1932296814539275
6 https://doi.org/10.1177/0264550512448452
7 https://doi.org/10.1366/11-06462
8 https://doi.org/10.1177/1057567714527390
9 https://doi.org/10.1177/1741134317728403
10 https://doi.org/10.1177/2051570714533474
11 https://doi.org/10.1177/0003702816638294
12 https://doi.org/10.1255/jnirs.1154
13 https://doi.org/10.1177/0954406211428397
14 https://doi.org/10.1177/1741134314554127
15 https://doi.org/10.1177/1077558716629039
16 https://doi.org/10.1177/203228441600700308
17 https://doi.org/10.1177/1057567712465755
18 https://doi.org/10.1177/1076029611426870
19 https://doi.org/10.1177/0004865815626768
20 https://doi.org/10.5301/maapoc.0000023
21 https://doi.org/10.5301/maapoc.0000013
22 https://doi.org/10.1177/1748895814538039
23 https://doi.o

In [123]:
# Append columns into one dataframe
df['Keywords'] = kw['Keywords']

In [124]:
len(df)

34

In [125]:
df.head()

,Author,Title,Pub_type,Journal,Year_publish,Doi_link,Abstract,Keywords
0,"Greenberg, Mary Anne",Serialization: Benefits Beyond Regulatory Comp...,NaN,Therapeutic Innovation & Regulatory Science,2014,https://doi.org/10.1177/2168479014553034,As pharmaceutical and biotech markets continue...,"Keywords serialization, compliance, counterfeit"
1,"Isles, Mike",What's in a Word? Falsified/Counterfeit/Fake M...,NaN,Medicine Access @ Point of Care,2017,https://doi.org/10.5301/maapoc.0000008,There is a rising tide of criminal activity to...,"Keywords Counterfeit, Fake medicine, Falsified"
2,"Carter, Alan",Improving the Drug Quality and Safety Net,NaN,Journal of Diabetes Science and Technology,2014,https://doi.org/10.1177/1932296814531100,"When large quantities of contaminated, subpote...","Keywords counterfeit, global, legislation, out..."
3,"Lavorgna, Anita",The online trade in counterfeit pharmaceutical...,NaN,European Journal of Criminology,2014,https://doi.org/10.1177/1477370814554722,There is widespread agreement that the Interne...,"Keywords Counterfeit, criminal opportunity, do..."
4,"Rojek, Chris","Counterfeit Commerce: Relations of Production,...",NaN,Cultural Sociology,2016,https://doi.org/10.1177/1749975516650233,Study of the consumption of counterfeit produc...,"Keywords counterfeit, commerce, counterfeit co..."


In [126]:
# download file w info about papers
df.to_csv('List_SAGE_pub_counterfeit.csv')
# ff2.to_csv('List_INFORMS_pub_online commerce retailing_v04.csv')

## Part 2. Download papers pdf
## Note: papers may duplicate within different keywords, so filter unique paper links

In [127]:
f = pd.read_csv("C:\Users\mariia.petryk/List_SAGE_pub_counterfeit.csv")
# data = f.readlines()

In [128]:
len(f)

17

In [129]:
j=0
for j in range(0,len(f['Doi_link'])):
# for j in range(0,5):
    try:
#         print(f['Doi_link'].iloc[j])
#         print f['Doi_link'].iloc[j].replace("/abs/", "/pdf/")
        link="http://journals.sagepub.com/doi/pdf/"+f['Doi_link'].iloc[j][16:]
        print j
        print link
        r = requests.get(link, stream = True, headers=header)
        print r
        rep=[", ",","]
#         "/Users/mariiapetryk/Documents/UF/Young/Information security_Informs/"+
        filename=str(f['Author'].iloc[j].replace(", ", "_").replace(",", "_"))+"_"+str(f['Year_publish'].iloc[j])+".pdf"
        print filename
        with open(filename,"wb") as pdf:
            pdf.write(r.content)
    except:
        pass
    sleep(10)

0
http://journals.sagepub.com/doi/pdf/10.1177/1477370814554722
<Response [200]>
Lavorgna_Anita_2014.pdf
1
http://journals.sagepub.com/doi/pdf/10.1177/1749975516650233
<Response [200]>
Rojek_Chris_2016.pdf
2
http://journals.sagepub.com/doi/pdf/10.1177/1057567714527390
<Response [200]>
Wilson_Jeremy M._Fenoff_Roy_2014.pdf
3
http://journals.sagepub.com/doi/pdf/10.1177/2051570714533474
<Response [200]>
Viot_Catherine_Le Roux_André_Krémer_Florence_2014.pdf
4
http://journals.sagepub.com/doi/pdf/10.1177/1077558716629039
<Response [200]>
Zullo_Andrew R._Howe_Chanelle J._Galárraga_Omar_2016.pdf
5
http://journals.sagepub.com/doi/pdf/10.1177/203228441600700308
<Response [200]>
Negri_Stefania_2016.pdf
6
http://journals.sagepub.com/doi/pdf/10.1177/1057567712465755
<Response [200]>
Heinonen_Justin A._Holt_Thomas J._Wilson_Jeremy M._2012.pdf
7
http://journals.sagepub.com/doi/pdf/10.1177/0004865815626768
<Response [200]>
Bandiera_Rhiannon_Marmo_Marinella_2016.pdf
8
http://journals.sagepub.com/doi/pdf/